## Install tf-gpu, imports e download dati, split train val

In [2]:
# tf ha come requisito che gast sia > 0.2, ma l'ultima versione di fast ha un 
# problema con autograph
!pip install gast==0.2.2
!pip install tensorflow-gpu==1.14

!wget http://www.itl.nist.gov/iaui/vip/cs_links/EMNIST/matlab.zip
!unzip matlab.zip

import os
from datetime import timedelta
import time
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp

tf.logging.set_verbosity(tf.logging.ERROR)  # or any {DEBUG, INFO, WARN, ERROR, FATAL}

tfd = tfp.distributions

# Controllo della versione di tf, perché da 1/10 quella di default è la 2.0.0
print(tf.__version__)  
print(tf.test.gpu_device_name())  # Controllo se colab sta usando la gpu

# Download dati
from keras.datasets import mnist
from scipy import io as spio
(x_train, y_train), (x_test, y_test) = mnist.load_data() # 60k - 10k

# Creo un asse aggiuntivo che rappresenta i colori, nelle immagini bio è 3 
# perché sono a colori, qua è 1 perché è a toni di grigio. Senza questo 
# si blocca sulla rete, cond2dflipout vuole che input gli passi un immagine con 
# 4 assi (1 per il batch), ma input riceve solo 3 assi da input_pipeline
x_train = np.expand_dims(x_train, axis=3)
x_test = np.expand_dims(x_test, axis=3)

emnist = spio.loadmat(os.path.join("matlab", "emnist-letters"))



     |████████████████████████████████| 377.0MB 65kB/s 
     |████████████████████████████████| 491kB 41.7MB/s 
     |████████████████████████████████| 3.2MB 30.1MB/s 
ERROR: tensorflow 1.15.0rc3 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 1.14.0 which is incompatible.
ERROR: tensorflow 1.15.0rc3 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 1.14.0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
--2019-10-21 13:21:47--  http://www.itl.nist.gov/iaui/vip/cs_links/EMNIST/matlab.zip
Resolving www.itl.nist.gov (www.itl.nist.gov)... 129.6.13.51, 2610:20:6b01:4::36
Connecting to www.itl.nist.gov (www.itl.nist.gov)|129.6.13.51|:80... connected.
HTTP request 

Using TensorFlow backend.


11493376/11490434 [==============================] - 0s 0us/step


In [3]:
def split_train_test(x, y, test_ratio):
  seed = np.random.RandomState(42)
  shuffled_indices  = seed.permutation(len(y))
  test_set_size = int(len(y) * test_ratio)
  test_indices = shuffled_indices[:test_set_size]
  train_indices = shuffled_indices[test_set_size:]
  
  return (x[train_indices], y[train_indices]), (x[test_indices], y[test_indices])

print(len(x_train))
(x_train, y_train), (x_val, y_val) = split_train_test(x_train, y_train, 0.15) 
print(len(x_train))

60000
51000


## Parametri

In [0]:
learning_rate = default=0.0001  # Initial learning rate
epochs = 500  #Number of epochs to train for
batch_size = 500  #Batch size
eval_freq = 400  # Frequency at which to validate the model
num_monte_carlo = 50  # Network draws to compute predictive probabilities

# kernel_posterior_scale_mean = -9.0  # Initial kernel posterior mean of the scale (log var) for q(w)
# kernel_posterior_scale_constraint =0.2  # kernel constraint for the scale (log var) of q(w).")

# kl_annealing = 50 #Epochs to anneal the KL term (anneals from 0 to 1)

IMAGE_SHAPE = [28, 28, 1]

model_dir = os.path.join(os.getenv("TEST_TMPDIR", "/tmp"),
                         "bayesian_neural_network/")  # Directory to put the model's fit


## Input pipeline

In [0]:
def build_input_pipeline(x_train, x_test, y_train, y_test,
                         batch_size):
  """Build an Iterator switching between train and heldout data."""
  
  # Siccome questo dataset è piccolo posso permettermi di dare all'heldout 
  # l'intero set come batch, inoltre non faccio la normalizzazione.
  valid_size = len(y_test)

  # Converto in float e normalizzo
  x_train = x_train.astype("float32")
  x_test = x_test.astype("float32")

  x_train /= 255
  x_test /= 255

  print("x_train shape:" + str(x_train.shape))
  print(str(x_train.shape[0]) + " train samples")
  print(str(x_test.shape[0]) + " test samples")

  # Build an iterator over training batches.
  # Per avere poi la pmf ti serve che i labels siano int32
  training_dataset = tf.data.Dataset.from_tensor_slices(
      (x_train, np.int32(y_train)))
  # Non so se sovrascriverlo faccia la diff
  training_batches = training_dataset.shuffle(
      len(x_train), reshuffle_each_iteration=True).repeat().batch(batch_size)
  training_iterator = tf.compat.v1.data.make_one_shot_iterator(training_batches)

  ####################################################################################################
  # Non capisco perché 500 (è il valid size)... non dovrebbe essere len(x_test) ?

  # Build a iterator over the heldout set with batch_size=heldout_size,
  # i.e., return the entire heldout set as a constant.
  heldout_dataset = tf.data.Dataset.from_tensor_slices((x_test, np.int32(y_test)))
  heldout_batches = heldout_dataset.repeat().batch(valid_size)  # niente reshuffle
  heldout_iterator = tf.compat.v1.data.make_one_shot_iterator(heldout_batches)

  # Combine these into a feedable iterator that can switch between training
  # and validation inputs.
  handle = tf.compat.v1.placeholder(tf.string, shape=[])
  feedable_iterator = tf.compat.v1.data.Iterator.from_string_handle(
      handle, training_batches.output_types, training_batches.output_shapes)
  images, labels = feedable_iterator.get_next()

  return images, labels, handle, training_iterator, heldout_iterator

## Main

In [6]:
with tf.name_scope("directory"):
  if tf.io.gfile.exists(model_dir):
    tf.compat.v1.logging.warning(
        "Warning: deleting old log directory at {}".format(model_dir))
    tf.io.gfile.rmtree(model_dir)
  tf.io.gfile.makedirs(model_dir)
  
with tf.name_scope("Dataset"):
  (images, labels, handle,
  training_iterator,
  heldout_iterator) = build_input_pipeline(x_train, x_val, y_train, y_val,
                                            batch_size)

with tf.name_scope("BNN"):
  inputs = tf.keras.layers.Input(shape=IMAGE_SHAPE, dtype='float32')
  x = tfp.layers.Convolution2DFlipout(32, (5, 5), activation='relu', padding='same')(inputs)
  x = tf.keras.layers.MaxPooling2D((2, 2))(x)
  x = tfp.layers.Convolution2DFlipout(64, (5, 5), activation='relu', padding='same')(x)
  x = tf.keras.layers.MaxPooling2D((2, 2))(x)

  x = tf.keras.layers.Flatten()(x)
  x = tfp.layers.DenseFlipout(100, activation='relu')(x)
  outputs = tfp.layers.DenseFlipout(10, activation=None)(x)
  model = tf.keras.Model(inputs, outputs)

logits = model(images)

with tf.name_scope("loss"):
  # Distribuzione categorica delle p per ogni istanza
  labels_distribution = tfd.Categorical(logits=logits)

  # Perform KL annealing. The optimal number of annealing steps
  # depends on the dataset and architecture.
  # t = tf.compat.v2.Variable(0.0)  # Non ho idea del motivo per cui usi compat.v2, sembra però una normale variabile
  # kl_regularizer = t / (FLAGS.kl_annealing * len(x_train) / FLAGS.batch_size)
    
  
  # Compute the -ELBO as the loss. The kl term is annealed from 0 to 1 over
  # the epochs specified by the kl_annealing flag.

  # Calcolare la neg log likelihood in questo modo oppure con sparse cross 
  # entropy dà risultati simili, ma non uguali, probabilmente qualche differenza
  # è nella costruzione della distribuzione categorica ed estrazione della sua pmf
  log_likelihood = labels_distribution.log_prob(labels)  # PMF di ogni istanza
  neg_log_likelihood = -tf.reduce_mean(input_tensor=log_likelihood)
  kl = sum(model.losses) / len(x_train) # * tf.minimum(1.0, kl_regularizer)
  loss = neg_log_likelihood + kl
  
# Build metrics for evaluation. Predictions are formed from a single forward
# pass of the probabilistic layers. They are cheap but noisy
# predictions.
predictions = tf.argmax(input=logits, axis=1)

with tf.name_scope("train"):
  train_accuracy, train_accuracy_update_op = tf.metrics.accuracy(
      labels=labels, predictions=predictions)
  optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate)
  train_op = optimizer.minimize(loss)

with tf.name_scope("valid"):
  valid_accuracy, valid_accuracy_update_op = tf.metrics.accuracy(
      labels=labels, predictions=predictions)

init_op = tf.group(tf.global_variables_initializer(),
                  tf.local_variables_initializer())  # è un'op che esegue tutti i suoi input

# Resetto le variabili che riguardano l'acc sul validation set, non voglio fare
# la media con la passata (epoca o iterazione?) precedente, ma le voglio ricalcolare
# da 0 per vedere come le performance sono cambiate
# Per resettare basta reinizializzarle
stream_vars_valid = [
    v for v in tf.local_variables() if "valid/" in v.name]
reset_valid_op = tf.compat.v1.variables_initializer(stream_vars_valid)

saver = tf.train.Saver()  # Nodo di salvataggio


x_train shape:(51000, 28, 28, 1)
51000 train samples
9000 test samples


## Execution phase

In [7]:
with tf.Session() as sess:
  sess.run(init_op)

  # Run the training loop

  train_handle = sess.run(training_iterator.string_handle())
  heldout_handle = sess.run(heldout_iterator.string_handle())
  # Faccio girare tutto in modo da avere 1 epoca (len(x_train)/batch_size) per 
  # il numero di epoche che voglio
  training_steps = int(
      round(epochs * (len(x_train) / batch_size)))
  
  # Non mi preoccupo del numero di step, tanto ho generato dati infiniti nel 
  # dataset tramite repeat()
  for step in range(training_steps):
    _ = sess.run([train_op,
                  train_accuracy_update_op],
                  feed_dict={handle: train_handle})

    # Manually print the frequency
    # ad ogni multiplo di 100 valuto loss, accuracy e kl
    if step % 100 == 0:

      loss_value, accuracy_value, kl_value = sess.run(
            [loss, train_accuracy, kl], feed_dict={handle: train_handle})
      print(
            "Step: {:>3d} Loss: {:.3f} Accuracy: {:.3f} KL: {:.3f}".format(
                step, loss_value, accuracy_value, kl_value))
      save_path = saver.save(sess, "/content/prova.ckpt")


    if (step + 1) % eval_freq == 0:
    # Calculate validation accuracy
      sess.run(valid_accuracy_update_op, feed_dict={handle: heldout_handle})
      valid_value = sess.run(valid_accuracy, feed_dict={handle: heldout_handle})

      print(" ... Validation Accuracy: {:.3f}".format(valid_value))

      sess.run(reset_valid_op)

Step:   0 Loss: 24.168 Accuracy: 0.104 KL: 18.191
Step: 100 Loss: 20.222 Accuracy: 0.163 KL: 18.160
Step: 200 Loss: 19.250 Accuracy: 0.296 KL: 18.127
Step: 300 Loss: 18.797 Accuracy: 0.423 KL: 18.095
 ... Validation Accuracy: 0.816
Step: 400 Loss: 18.716 Accuracy: 0.514 KL: 18.061
Step: 500 Loss: 18.500 Accuracy: 0.579 KL: 18.027
Step: 600 Loss: 18.357 Accuracy: 0.626 KL: 17.992
Step: 700 Loss: 18.237 Accuracy: 0.663 KL: 17.956
 ... Validation Accuracy: 0.907
Step: 800 Loss: 18.267 Accuracy: 0.693 KL: 17.919
Step: 900 Loss: 18.107 Accuracy: 0.717 KL: 17.881
Step: 1000 Loss: 18.088 Accuracy: 0.737 KL: 17.843
Step: 1100 Loss: 17.989 Accuracy: 0.755 KL: 17.804
 ... Validation Accuracy: 0.939
Step: 1200 Loss: 17.875 Accuracy: 0.769 KL: 17.764
Step: 1300 Loss: 17.928 Accuracy: 0.782 KL: 17.723
Step: 1400 Loss: 17.840 Accuracy: 0.794 KL: 17.682
Step: 1500 Loss: 17.771 Accuracy: 0.804 KL: 17.640
 ... Validation Accuracy: 0.948
Step: 1600 Loss: 17.762 Accuracy: 0.813 KL: 17.597
Step: 1700 Loss

KeyboardInterrupt: ignored

## emnist

In [0]:
x_train = emnist["dataset"][0][0][0][0][0][0]
x_train = np.squeeze(x_train.astype(np.float32))
y_train = np.squeeze(emnist["dataset"][0][0][0][0][0][1])
x_test = emnist["dataset"][0][0][1][0][0][0]
x_test = x_test.astype(np.float32)
y_test = np.squeeze(emnist["dataset"][0][0][1][0][0][1])
x_train /= 255
x_test /= 255
x_train = x_train.reshape(x_train.shape[0], 28, 28, order="A")
x_test = x_test.reshape(x_test.shape[0], 28, 28, order="A")

# fondamentale normalizzare, siccome però è un dataset semplice non devi toccare
# la media, ma basta riportare tutto tra 0 e 1

fake_labels = np.zeros((len(x_test), 10))

## Prove

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!cp /content/prova.ckpt.data-00000-of-00001 gdrive/'My Drive'/Dataset_progetto
!cp /content/prova.ckpt.index gdrive/'My Drive'/Dataset_progetto
!cp /content/prova.ckpt.meta gdrive/'My Drive'/Dataset_progetto
!cp /content/checkpoint gdrive/'My Drive'/Dataset_progetto


In [9]:
with tf.Session() as sess:
  sess.run(init_op)
  saver.restore(sess, "/content/gdrive/My Drive/Dataset_progetto/prova.ckpt")

  # Run the training loop

  train_handle = sess.run(training_iterator.string_handle())
  heldout_handle = sess.run(heldout_iterator.string_handle())
  # Faccio girare tutto in modo da avere 1 epoca (len(x_train)/batch_size) per 
  # il numero di epoche che voglio
  training_steps = int(
      round(epochs * (len(x_train) / batch_size)))
  

    # Calculate validation accuracy
  sess.run(valid_accuracy_update_op, feed_dict={handle: heldout_handle})
  valid_value = sess.run(valid_accuracy, feed_dict={handle: heldout_handle})

  print(" ... Validation Accuracy: {:.3f}".format(valid_value))



 ... Validation Accuracy: 0.963


In [12]:
#os.path.exists(content/gdrive/'My Drive'/Dataset_progetto/Saver)
os.path.exists("/content/gdrive/My Drive/Dataset_progetto/Saver")
os.path.exists("/content/gdrive/My Drive/Dataset_progetto")

True